# CISCO设备 ARP 配置
* 理解netmiko的主要API
* https://ktbyers.github.io/netmiko/docs/netmiko/base_connection.html

In [1]:
import re
from netmiko import ConnectHandler

class playARP():
    def __init__(self, connect_info_dict):
        self.net = ConnectHandler(**connect_info_dict)
        self.ip_pattern = r"^(([01]{0,1}\d{0,1}\d|2[0-4]\d|25[0-5])\.){3}([01]{0,1}\d{0,1}\d|2[0-4]\d|25[0-5])$"
        self.mac_pattern = r"^([A-F0-9]{4}\.[A-F0-9]{4}\.[A-F0-9]{4}|[a-f0-9]{4}\.[a-f0-9]{4}\.[a-f0-9]{4})$"
        self.type_pattern = r"^(arpa|sap|smds|snap|srp-a|srp-b)$"
        self.add_ip_list = []
        
    def _connect_device(self):
        if not self.net.is_alive():
            self.net = ConnectHandler(**connect_info_dict)
            #print(f"连接成功，当前提示符为{self.net.find_prompt()}")
        
    def get_arp(self):
        self._connect_device()
        print(self.net.send_command("show ip arp"))
        
    def add_arp(self, ip, mac, arp_type="arpa"):
        self._connect_device()
        if not re.search(self.ip_pattern, ip):
            print("IP地址格式错误")
        elif not re.search(self.mac_pattern, mac):
            print("MAC地址格式错误")
        elif not re.search(self.type_pattern, arp_type):
            print("ARP类型错误")
        else:
            self.net.send_config_set(f"arp {ip} {mac} {arp_type}")
            self.get_arp()
            self.add_ip_list.append(ip)
            print("配置成功")
            
    def del_arp(self, ip):
        self._connect_device()
        if not re.search(self.ip_pattern, ip):
            print("IP地址格式错误")
        elif ip not in self.add_ip_list:
            print("此IP地址并没有通过<add_arp>添加过")
        else:
            self.net.send_config_set(f"no arp {ip}")
            self.get_arp()
            self.add_ip_list.remove(ip)

In [2]:
connect_info_dict= {
    "device_type": "cisco_ios",
    "host": "192.168.1.50",
    "username": "ciscouser",
    "secret": "cisco123",
    "password": "cisco@123",
}

In [5]:
my_arp = playARP(connect_info_dict)
my_arp.get_arp()


Protocol  Address          Age (min)  Hardware Addr   Type   Interface
Internet  192.168.1.50            -   c201.36ee.0000  ARPA   FastEthernet0/0
Internet  192.168.1.1             0   30cc.218d.2220  ARPA   FastEthernet0/0
Internet  192.168.1.7             0   4845.20ba.1512  ARPA   FastEthernet0/0
Internet  192.168.1.6             0   0c82.6830.09f6  ARPA   FastEthernet0/0


In [14]:
my_arp.add_arp("1.1.1.1", "aaaa.bbbb.cccc", "arpa")


Protocol  Address          Age (min)  Hardware Addr   Type   Interface
Internet  1.1.1.1                 -   aaaa.bbbb.cccc  ARPA  
Internet  192.168.1.50            -   c201.0f5e.0000  ARPA   FastEthernet0/0
Internet  192.168.1.7             0   4845.20ba.1512  ARPA   FastEthernet0/0
Internet  192.168.1.6             0   0c82.6830.09f6  ARPA   FastEthernet0/0
配置成功


In [17]:
my_arp.del_arp("1.1.1.1")


Protocol  Address          Age (min)  Hardware Addr   Type   Interface
Internet  192.168.1.50            -   c201.0f5e.0000  ARPA   FastEthernet0/0
Internet  192.168.1.7             0   4845.20ba.1512  ARPA   FastEthernet0/0
Internet  192.168.1.6             0   0c82.6830.09f6  ARPA   FastEthernet0/0


In [6]:
my_arp.net.check_enable_mode()

True

In [7]:
my_arp.net.exit_enable_mode()

'disable\r\nR1>'

In [8]:
my_arp.net.find_prompt()

'R1>'

In [12]:
my_arp.add_arp("2.2.2.2", "aaaa.bbbb.cccc")


Protocol  Address          Age (min)  Hardware Addr   Type   Interface
Internet  2.2.2.2                 -   aaaa.bbbb.cccc  ARPA  
Internet  192.168.1.50            -   c201.36ee.0000  ARPA   FastEthernet0/0
Internet  192.168.1.1             1   30cc.218d.2220  ARPA   FastEthernet0/0
Internet  192.168.1.7             0   4845.20ba.1512  ARPA   FastEthernet0/0
Internet  192.168.1.6             0   0c82.6830.09f6  ARPA   FastEthernet0/0
配置成功


In [11]:
my_arp.net.find_prompt()

'R1#'

In [10]:
my_arp.net.enable()

'enable\r\nPassword: \r\nR1#'

In [13]:
my_arp.del_arp("2.2.2.2")


Protocol  Address          Age (min)  Hardware Addr   Type   Interface
Internet  192.168.1.50            -   c201.36ee.0000  ARPA   FastEthernet0/0
Internet  192.168.1.1             1   30cc.218d.2220  ARPA   FastEthernet0/0
Internet  192.168.1.7             0   4845.20ba.1512  ARPA   FastEthernet0/0
Internet  192.168.1.6             0   0c82.6830.09f6  ARPA   FastEthernet0/0
